<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [1]:
!pip install dotenv

## Importações

In [36]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
from dotenv import load_dotenv

## Pegando as chaves

In [37]:
load_dotenv()
PUBLIC_KEY = os.getenv("Public_Key")
PRIVATE_KEY = os.getenv("Private_Key")

## Endpoints e Parâmetros

In [38]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

end1 = "https://gateway.marvel.com/v1/public/characters"
end2 = "https://gateway.marvel.com/v1/public/comics"
end3 = "https://gateway.marvel.com/v1/public/books"
end4 = "https://gateway.marvel.com/v1/public/series"
offset = 0
params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 10,
    "offset": offset
}

## Requisição

In [5]:
response = requests.get(end1, params=params)

response
data = response.json()

df = pd.DataFrame(data['data']['results'])
df.dtypes
# df_extraido = pd.json_normalize(df['data']['results'])
# df_extraido2 = pd.DataFrame(df['data']['results'])

,0
id,int64
name,object
description,object
modified,object
thumbnail,object
resourceURI,object
comics,object
series,object
stories,object
events,object


## Exibição

In [6]:
# dsa = df_extraido['name']
df = df[['name', 'id', 'description']]
con = sqlite3.connect('Marvel.db')
df.to_sql("characters", con, if_exists="replace", index=False)
con.close()

## Consultando o Banco de Dados

In [7]:
con2 = sqlite3.connect('Marvel.db')
df2 = pd.read_sql_query("SELECT * FROM characters", con2)
con2.close()
df2

,name,id,description
0,3-D Man,1011334,
1,A-Bomb (HAS),1017100,Rick Jones has been Hulk's best bud since day ...
2,A.I.M.,1009144,AIM is a terrorist organization bent on destro...
3,Aaron Stack,1010699,
4,Abomination (Emil Blonsky),1009146,"Formerly known as Emil Blonsky, a spy of Sovie..."
5,Abomination (Ultimate),1016823,
6,Absorbing Man,1009148,
7,Abyss,1009149,
8,Abyss (Age of Apocalypse),1010903,
9,Adam Destine,1011266,


## Consumindo todas as Series


In [47]:
  end4 = "https://gateway.marvel.com/v1/public/series"
  offset = 0
  params = {
      "apikey": PUBLIC_KEY,
      "ts": ts,
      "hash": hash_md5,
      "limit": 10,
      "offset": offset
  }
  arraySeries = []
  response = requests.get(end4, params=params)
  while response.status_code == 200:
    arraySeries.append(response.json())
    offset += 100
    params['offset'] = offset
    response = requests.get(end4, params=params)
  print(response.json())
  print(response.status_code)
  return arraySeries

## Adicionando o banco de dados

In [44]:
def addDB(arraySeries):
  con = sqlite3.connect('Marvel.db')
  for i in arraySeries:
    df = pd.DataFrame(pd.DataFrame(i['data']['results']))
    df = df[['id', 'title', 'description', 'startYear', 'endYear', 'rating', 'type']]
    df.to_sql("series", con, if_exists="replace", index=False)
  con.close()

In [48]:
array = consumeSeries(PUBLIC_KEY, ts, hash)
array

{'message': 'Forbidden'}
403


[]